In [2]:
#libraries / dependencies
import glob
from preprocessor import api as tweet_preprocessor

import numpy as np
import math
import random

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

from pyspark.sql.functions import col, udf, to_timestamp, lit, to_timestamp, when, rand
from pyspark.sql.types import IntegerType, LongType, DoubleType, StringType, ArrayType
from pyspark.ml.feature import Normalizer, StandardScaler, MinMaxScaler, VectorAssembler

from pyspark import StorageLevel
from pyspark.accumulators import AccumulatorParam

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy


#thread depencency
from pyspark import InheritableThread

import findspark

findspark.init()
findspark.find()



# initialize the number of epochs to train for, batch size, and
# initial learning rate
EPOCHS = 25
BS = 64
INIT_LR = 1e-3

mse = MeanSquaredError()
cce = CategoricalCrossentropy()
opt = Adam(learning_rate=INIT_LR)



#Dataset location

#Local
bot_tweets_dataset_path = 'F://TwitterBotDataset//tweet_dataset_full//bot_tweets//'
genuine_tweets_dataset_path = 'F://TwitterBotDataset//tweet_dataset_full//genuine_tweets//'

#S3
# bot_tweets_dataset_path = 's3://tweet-dataset/bot_tweets' #'F://TwitterBotDataset//tweet_dataset_small//bot_tweets//'
# genuine_tweets_dataset_path = 's3://tweet-dataset/genuine_tweets' #'F://TwitterBotDataset//tweet_dataset_small//genuine_tweets//'

#turn a line of text into d dimentional vector 
GLOVE_DIR = ""

#all columns
BOT_COLUMNS = ['id','text','source','user_id','truncated','in_reply_to_status_id', 
               'in_reply_to_user_id','in_reply_to_screen_name', 'retweeted_status_id',
               'geo','place','contributors','retweet_count', 'reply_count','favorite_count',
               'favorited', 'retweeted','possibly_sensitive','num_hashtags','num_urls',
               'num_mentions','created_at','timestamp','crawled_at', 'updated']

GENUINE_COLUMNS = ['id','text','source','user_id','truncated','in_reply_to_status_id', 
                   'in_reply_to_user_id','in_reply_to_screen_name', 'retweeted_status_id',
                   'geo','place','contributors','retweet_count', 'reply_count','favorite_count',
                   'favorited', 'retweeted','possibly_sensitive','num_hashtags','num_urls',
                   'num_mentions','REMOVE_IT', 'created_at','timestamp','crawled_at', 'updated',]

#feature used for bot detection
COLUMN_NAMES = ['text', 'retweet_count', 'reply_count', 'favorite_count',
                'num_hashtags', 'num_urls', 'num_mentions']


# #configure spark
# conf = SparkConf()
# conf.setMaster("local[8]").setAppName("ml_account_ base_session")
# conf.set("spark.executor.instances", 4)
# conf.set("spark.executor.cores", 4)
# conf.set("spark.driver.memory", 4)
# sc = SparkContext(conf=conf)

# # for spark-submit
# spark = SparkSession.builder.appName('ml_account_ base_session').getOrCreate()
# spark

# for local build
# spark = SparkSession.builder.appName('ml_account_ base_session').getOrCreate()


#for local multi thread
conf = SparkConf()
conf.setMaster("local[20]").setAppName("data_parallel_processing_session")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# read dataset from csv
def read_dataset():
    bot_tweets = spark.read.csv(bot_tweets_dataset_path, header = True, inferSchema = True).limit(1000)
    genuine_tweets = spark.read.csv(genuine_tweets_dataset_path, header = True, inferSchema = True).limit(1000)
    
    #df = inputDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
#     bot_tweets = bot_tweets.persist(StorageLevel.MEMORY_ONLY)
#     genuine_tweets = genuine_tweets.persist(StorageLevel.MEMORY_ONLY)
    
#     print(len(bot_tweets.collect()), len(genuine_tweets.collect()))
    return bot_tweets, genuine_tweets

def set_column_name(df, column_names):
    df = df.toDF(*column_names)
#     print(len(df.collect()))
    return df

def remove_column_miss_match(df):
    ## dataset have diffrent number of columns
    ## column name of dataframe
    column_name = [cname for cname, tp in df.dtypes]
#     len(df.collect()), len(df.dtypes)
    #column_name

    #Number of column is diffrent for bot and genuine tweets data

    #genuine_tweets_df = genuine_tweets_df.toDF(*column_name)
    df = set_column_name(df, GENUINE_COLUMNS)
#     print(len(df.collect()))
    
    df = df.drop('REMOVE_IT') # remove 5th column from end
    #update column name according to 
    df = set_column_name(df, BOT_COLUMNS)
#     print(len(df.collect()))
    return df


def remove_type_miss_match(df):
    # Same column has diffrent data type. So make data type same for every column
    genuine_tweets_df = df.withColumn("id",col("id").cast(IntegerType())) \
                                    .withColumn("favorite_count",col("favorite_count").cast(LongType())) \
                                    .withColumn("favorited",col("favorited").cast(IntegerType()))
    return df


def resize_combine_data(bot_tweets_df, genuine_tweets_df):
    ## only keep the required column from the dataframe
    bot_tweets_df = bot_tweets_df.select(*COLUMN_NAMES)
    genuine_tweets_df = genuine_tweets_df.select(*COLUMN_NAMES)
    
#     print(len(bot_tweets_df.collect()), len(genuine_tweets_df.collect()))

    ## add BotOrNot column
    bot_tweets_df = bot_tweets_df.withColumn('BotOrNot', lit(1))
    genuine_tweets_df = genuine_tweets_df.withColumn('BotOrNot', lit(0))

    #combine clean and bot accounts data togather
    tweets_df = bot_tweets_df.union(genuine_tweets_df)

    # shuffle dataset
    tweets_df = tweets_df.orderBy(rand())

#     print(len(tweets_df.collect()))
    
    return tweets_df

text_process_udf = udf(lambda x : tweet_preprocessor.tokenize(x), StringType())
def preprocess_data(df):
    df = df.withColumn('text', text_process_udf(df.text))
    df = df.withColumn("retweet_count",col("retweet_count").cast(DoubleType()))
    df = df.withColumn("reply_count",col("reply_count").cast(DoubleType()))
    df = df.withColumn("favorite_count",col("favorite_count").cast(DoubleType()))
    df = df.withColumn("num_hashtags",col("num_hashtags").cast(DoubleType()))
    df = df.withColumn("num_urls",col("num_urls").cast(DoubleType()))
    df = df.withColumn("num_mentions",col("num_mentions").cast(DoubleType()))
    
    return df


def makeGloveWordEmbedder(glove_path):
    embedding_dict = {}
    with open(glove_path, 'r', encoding="utf-8") as glove_file:
        for line in glove_file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embedding_dict[word] = vector
            
    return embedding_dict  


# # Test GLoVE result
# glove_word2vec_embedder["google"]

 ##create word embedding GLoVE model dictionary. Use pre trained model
text_feature_dimention = 25
# glove_word2vec_embedder = makeGloveWordEmbedder(GLOVE_DIR + "glove.twitter.27B.25d.txt")


#Give a word and get that word representing feature vector of dimention 25 from embedding dictionary
def word2vec(word_dict=None, word=None, dim=25):
    default_vector = np.zeros(dim)
    
    if word_dict is None or word is None:
        return default_vector
    
    word_vector = word_dict.get(word)
    
    if word_vector is None:
        return default_vector
    return word_vector

# # test a word representing feature vector
# word_vector = word2vec(glove_word2vec_embedder, "tweet", text_feature_dimention)
# print(type(word_vector), word_vector)


#----------------LSTM Model----------------------

#create a 1 layer LSTM model
#input dimention: 3d (1,7,25) input_sample = [[[1,...,25]],..,[1,...,25]]
#output dimention: 1d (1,32) output_sample [[1,2,3,,,,32]]
def lstm_model(output_dim):
    model = LSTM(output_dim, return_sequences=False, return_state=False)
    return model

def reset_lstm_model(model):
    model.resate_states() # stateful=True is required for reset states

# create LSTM model of output dimention 32. Model output feature vector will be of 32 dimention vector
lstm = lstm_model(32) 

# convver a sentence to a feature vector using LSTM(RNN) model
def sent2vec(sent):
    words = sent.split(' ')
    word_vectors = np.array([])
    count = 0;
    for word in words:
        word_vector = word2vec(broadcast_glove_dict.value, word)
#         print("word dim: {}".format(len(word_vector)))
        if word_vectors.size == 0:
            word_vectors = np.array([word_vector])
        else:
            word_vectors = np.vstack([word_vectors, word_vector])
        count = count + 1
    
#     print("Input feature vector shape before reshape(2D): {}".format(word_vectors.shape))
        
    input_feature_vectors = np.reshape(word_vectors, (1, count, text_feature_dimention))
#     print("Input feature vector shape after reshape(3d): {}".format(input_feature_vectors.shape))
#     print("LSTM requirs 3d shape inputs [batch, timesteps, feature]")
    output_vector = lstm(input_feature_vectors)
#     lstm.reset_states() # stateful = True is required for reset

#     print("result vector shape: {}".format(output_vector.shape))
#     print("Last input was: {}".format(input_feature_vectors[0][-1]))
#     print("output result: {}".format(output_vector))
    
    # (tensore --> numpy 0bject --> numpy.array --> array/list/ArrayType)
    return output_vector.numpy()[0].tolist() 
    
## For Testing sentence to vector convertion
# sent = "Twitter is a large social media network"
# res_vector = sent2vec(sent)
# type(res_vector), res_vector


# text string --> vector 32 dimention
sent_to_vector_udf = udf(lambda x : sent2vec(x), ArrayType(DoubleType()))
def processTextColumn(df, column_name, new_column_name):
    df = df.withColumn(new_column_name, sent_to_vector_udf(col(column_name)))
    return df

def sentEmbeddingGLoVE_LSTM(df):
    
    text_updated_column = 'text_features'
    updated_df = processTextColumn(df, "text", text_updated_column)

#     print(len(updated_df.collect()), type(updated_df), updated_df.printSchema()) 
    
    return updated_df



def assembleColumns(tweets_df):
    columns = ['retweet_count', 'reply_count', 'favorite_count',
               'num_hashtags' ,'num_urls', 'num_mentions', 'BotOrNot']

    tweets_df = tweets_df.select(*columns, 
                          tweets_df.text_features[0], tweets_df.text_features[1], tweets_df.text_features[2],tweets_df.text_features[3], tweets_df.text_features[4],
                          tweets_df.text_features[5], tweets_df.text_features[6], tweets_df.text_features[7],tweets_df.text_features[8], tweets_df.text_features[9], 
                          tweets_df.text_features[10], tweets_df.text_features[11], tweets_df.text_features[12],tweets_df.text_features[13], tweets_df.text_features[14],
                          tweets_df.text_features[15], tweets_df.text_features[16], tweets_df.text_features[17],tweets_df.text_features[18], tweets_df.text_features[19],
                          tweets_df.text_features[20], tweets_df.text_features[21], tweets_df.text_features[22],tweets_df.text_features[23], tweets_df.text_features[24],
                          tweets_df.text_features[25], tweets_df.text_features[26], tweets_df.text_features[27],tweets_df.text_features[28], tweets_df.text_features[29],
                          tweets_df.text_features[30], tweets_df.text_features[31])


#     print(tweets_df.columns, len(tweets_df.collect()), tweets_df.printSchema())

    #remove 

    feature_columns = ['retweet_count','reply_count','favorite_count','num_hashtags','num_urls','num_mentions',
                       'text_features[0]','text_features[1]', 'text_features[2]','text_features[3]','text_features[4]',
                       'text_features[5]','text_features[6]','text_features[7]', 'text_features[8]','text_features[9]',
                       'text_features[10]','text_features[11]','text_features[12]','text_features[13]','text_features[14]',
                       'text_features[15]','text_features[16]','text_features[17]','text_features[18]','text_features[19]',
                       'text_features[20]','text_features[21]','text_features[22]', 'text_features[23]', 'text_features[24]',
                       'text_features[25]','text_features[26]','text_features[27]', 'text_features[28]', 'text_features[29]',
                       'text_features[30]','text_features[31]']


    tweets_df = tweets_df.na.fill(value=0.0 ,subset= feature_columns)
    feature_assembler = VectorAssembler(inputCols = feature_columns, outputCol = 'independent_features')

    tweets_updated_df = feature_assembler.transform(tweets_df)

    #check
#     num = len(tweets_updated_df.collect())
#     print(num, type(tweets_updated_df), tweets_updated_df.printSchema())

    #remove unnecessary columns
    tweets_updated_df = tweets_updated_df.drop(*feature_columns)
    
    return tweets_updated_df


def to_nparray_list(df, column_name):
    rows = df.select(column_name).collect()
    lists = [x[column_name] for x in rows]
    nparr = np.array(lists)
    
    return nparr

def to_nparray_dataset(df, feature_column, target_column):
#     list(df.select('col_name').toPandas()['col_name']) 
#     feature = list(df.select(feature_column).toPandas()[feature_column])
#     target = list(df.select(target_column).toPandas()[target_column])
    feature = [row[0] for row in list(df.select(feature_column).toLocalIterator())]
    target = [row[0] for row in list(df.select(target_column).toLocalIterator())]
        
    return np.array(feature), np.array(target)    

def partition_dataset(df):
    train_df, test_df = df.randomSplit([0.80, 0.20])
#     print(len(train_df.collect()), len(test_df.collect()))

    # features --> 'BotOrNot'
#     X_train = train_df.drop('BotOrNot')
#     y_train = train_df.select('BotOrNot')
#     X_test = test_df.drop('BotOrNot')
#     y_test = test_df.select('BotOrNot')
    

    #checkpoint
#     print(len(X_train.collect()), len(y_train.collect()))
#     print(len(X_test.collect()), len(y_test.collect()))

#     X_train = to_nparray_list(X_train, 'independent_features')
#     y_train = to_nparray_list(y_train, 'BotOrNot')
#     X_test = to_nparray_list(X_test, 'independent_features')
#     y_test = to_nparray_list(y_test, 'BotOrNot')
    train_df = train_df.cache()
    test_df = test_df.cache()

    train_X,  train_Y = to_nparray_dataset(train_df, 'independent_features', 'BotOrNot')
    test_X, test_Y = to_nparray_dataset(test_df, 'independent_features', 'BotOrNot')

    
    return train_X, train_Y, test_X, test_Y # return type: numpy.array

'''
def distributedTrainingGradients(df, feature_column, target_column, n_splits):
    print(df.count())
    each_len = df.count() // n_splits
    gradients = []
    ##split dataset into 'n_splits' part
    copy_df = df
    for i in range(n_splits):
        print(i)
        temp_df = copy_df.limit(each_len)
        copy_df = copy_df.subtract(temp_df)
        
        X = temp_df.select(feature_column)
        Y = temp_df.select(target_column)
        X_np = to_nparray_list(X, feature_column)
        Y_np = to_nparray_list(Y, target_column)
        
        grad = step(X_np, Y_np)
        gradients.append(grad)
        print(temp_df.count())
        
    return gradients
'''

def generateGradient(X, Y):
    gd = step(X, Y)
    return gd

def step(X, Y):
    print("Input count: {}, {}".format(len(X), len(Y)))
    #keep track of gradients
    with tf.GradientTape() as tape:
        curr_model = get2DenseLayeredModel(38)
        #make a prediction using model
        predict = curr_model(X)
        #calculate loss
        loss = mse(Y, predict)
    #calculate the gradient
    gd = tape.gradient(loss, curr_model.trainable_variables)
    
    # return the gradient to train final model
    return gd



'''
def stepEPOCH(X, y):
    with tf.GradientTape() as tape:
        curr_model = getDLModel()
        for i in range(EPOCHS):
            #make a prediction using model
            predict = curr_model(X)
            #calculate loss
            loss = cce(y, predict)
            print("{}: {}".format(i, loss))
            opt.apply_gradients(zip(grad, curr_model.trainable_variables))
            
    gradient = tape.gradient(loss, curr_model.trainable_variables)
    
    return gradient

'''
    
    
        

## create model
def get2DenseLayeredModel(input_dim):
    model = Sequential()
    model.add(Dense(500, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

def model_evaluation(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train,
              batch_size=64,
              epochs=10,
              validation_data=(X_test, y_test))
    score, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test accuracy:', acc)

gds = []
def worker_task(bot_tweets_df, genuine_tweets_df):
#     #cache df
    bot_tweets_df = bot_tweets_df.cache()
    genuine_tweets_df = genuine_tweets_df.cache()
    
    print("#bot_tweets: {} #gen_tweets: {}".format(bot_tweets_df.count(), genuine_tweets_df.count()))
    
    ##clean data / remove unwanted column
    if len(bot_tweets_df.columns) == 26:
        bot_tweets_df = remove_column_miss_match(bot_tweets_df)
    else:
        bot_tweets_df = set_column_name(bot_tweets_df, BOT_COLUMNS)
        
    if len(genuine_tweets_df.columns) == 26:
        genuine_tweets_df = remove_column_miss_match(genuine_tweets_df)
    else:
        genuine_tweets_df = set_column_name(genuine_tweets_df, BOT_COLUMNS)
    
    
    bot_tweets_df = remove_type_miss_match(bot_tweets_df)
    genuine_tweets_df = remove_type_miss_match(genuine_tweets_df)
    
#     print(len(bot_tweets_df.collect()), len(genuine_tweets_df.collect()))
    
    ##preprocess data
    tweets_df = resize_combine_data(bot_tweets_df, genuine_tweets_df)
    tweets_df = preprocess_data(tweets_df)
    
#     print(len(tweets_df.collect()))
#     print(tweets_df.columns)
    
    ##text embedding using GLoVE & LSTM
    ## Word Embedding
    tweets_df = sentEmbeddingGLoVE_LSTM(tweets_df)
    
    ## Assable multiple colu,ms to create feature vector
    tweets_updated_df = assembleColumns(tweets_df)
#     print(len(tweets_updated_df.collect()), tweets_updated_df.columns)

#     tweets_updated_df = tweets_updated_df.cache()
    
    X, Y = to_nparray_dataset(tweets_updated_df, 'independent_features', 'BotOrNot')
    
    curr_gd = generateGradient(X, Y)
    
    accu_count.add(1)
    accu_gd.add(curr_gd)
    
#     gds.append(curr_gd)
    print("Curr gd: {}".format(curr_gd))
    print(":: OK")
    
#     ##Create Dense Model
    
#     ##device data into test and train parts
#     X_train, y_train, X_test, y_test = partition_dataset(tweets_updated_df)
    
#     ## ml model train and validation
#     model = get2DenseLayeredModel(38)
    
#     ##evaluate model
#     model_evaluation(model, X_train, y_train, X_test, y_test)
    
#     # save gradient to hdfs/s3


# broadcast glove word wmbedder to all task
def broadcastGloveDict():
    print("broadcast glove")
    glove_dict = makeGloveWordEmbedder(GLOVE_DIR + "glove.twitter.27B.25d.txt")
    broadcast_glove_dict = sc.broadcast(glove_dict)

glove_dict = makeGloveWordEmbedder(GLOVE_DIR + "glove.twitter.27B.25d.txt")
broadcast_glove_dict = sc.broadcast(glove_dict)



# accumulate partial train gradients
class TensorAccumulator(AccumulatorParam):
    def zero(self, value):
        return None
    
    def addInPlace(self, val1, val2):
        if val1 is None:
            val1 = val2
        else:
            val1 += val2
        return val1

accu_gd = spark.sparkContext.accumulator(None, TensorAccumulator())
accu_count = spark.sparkContext.accumulator(0)
    
def accumulateGradient():
    print("accumulate gds")

def ApplicationJob():
    bot_tweets_df, genuine_tweets_df = read_dataset()
    
#     broadcastGloveDict()
    # split dataset for parallel data training
    num_of_thread = 10
    split_weight = 1.0 / num_of_thread
    weights = [split_weight] * num_of_thread
    bot_dfs = bot_tweets_df.randomSplit(weights, seed = 71)
    gen_dfs = bot_tweets_df.randomSplit(weights, seed = 71)
    

    for idx in range(num_of_thread):
        thread = InheritableThread(target = worker_task, kwargs={'bot_tweets_df': bot_dfs[idx], 
                                                                'genuine_tweets_df': gen_dfs[idx]})
        thread.start()
        thread.join()
    
    print("#gd: {}".format(accu_count.value))
    print("Sum of gd: {}".format(accu_gd))
#     ## single worker or multiple worker
#     worker_task(bot_tweets_df, genuine_tweets_df)
    

if __name__ == '__main__':
    ##load dara
    ApplicationJob()
    spark.stop()

#bot_tweets: 105 #gen_tweets: 105
Input count: 210, 210
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00, -1.1687484e-06,  0.0000000e+00, ...,
        -3.6305224e-07,  0.0000000e+00,  1.6548127e-07],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -1.1311359e-06,  0.0000000e+00, ...,
        -9.6262352e-07,  8.0354221e-07,  0.0000000e+00],
       ...,
       [-1.9126048e-07,  4.3795867e-06, -7.4880290e-08, ...,
         1.0497497e-06, -5.8225637e-06, -1.1717495e-06],
       [-1.9072915e-07,  5.9109561e-07, -4.3454683e-08, ...,
         2.2985587e-06, -3.3951014e-06,  1.0577437e-06],
       [-2.2686422e-09, -2.6719797e-06, -4.4141423e-08, ...,
         5.9597397e-07, -1.6506146e-07, -1.8497883e-07]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([ 3.10155974e-06, -1.16721128e-06,  2.59644025e-07,  1.94834479e-06,
        1.24073762e-04

#bot_tweets: 92 #gen_tweets: 92
Input count: 184, 184
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.66507070e-05, -8.52820904e-07, ...,
         0.00000000e+00, -6.54875930e-06,  0.00000000e+00],
       ...,
       [ 8.57312989e-06,  2.94889196e-06, -1.46976308e-05, ...,
         1.58533987e-06,  2.98247851e-05, -1.20403784e-05],
       [ 4.78583479e-06,  3.35398636e-06, -1.41993005e-05, ...,
         6.41954159e-07,  2.40493428e-05, -1.43689385e-05],
       [ 4.44033367e-06,  9.72788257e-07, -4.30947557e-06, ...,
         5.36018149e-07,  5.64747279e-06, -4.84653992e-06]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([-1.80722796e-04, -1.66507070e-05,  9.12522228e-05,  7.95

#bot_tweets: 100 #gen_tweets: 100
Input count: 200, 200
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-3.29420754e-06,  2.22842959e-06, -2.10305507e-06, ...,
        -8.88933300e-07, -2.08419897e-06, -3.96708720e-06],
       [-8.53783058e-06,  9.61411843e-06, -2.13719723e-05, ...,
        -1.83474401e-06, -9.18398200e-06,  2.05112974e-06],
       [-6.82724988e-07,  1.81014116e-07, -9.08424568e-07, ...,
         1.05176255e-07, -9.59657427e-07, -1.97583859e-06]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([ 2.52400896e-05, -6.63138708e-05,  8.34414095e-05,  2.

#bot_tweets: 91 #gen_tweets: 91
Input count: 182, 182
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-6.3269708e-06,  1.2086521e-05, -1.1683340e-05, ...,
        -1.1928252e-05, -6.5962222e-06, -3.1964926e-05],
       [-7.0315209e-06,  1.5332562e-05, -1.0711052e-05, ...,
        -2.1420838e-05, -7.7637296e-06, -4.5569232e-05],
       [ 1.2815235e-08, -9.4177303e-06, -2.7320484e-06, ...,
         1.2761933e-05,  1.0787233e-05,  3.8942846e-05]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([-7.62034251e-05,  2.25110896e-04, -1.07878732e-04,  3.57344870e-05,
        3.66328779e-04, 

#bot_tweets: 107 #gen_tweets: 107
Input count: 214, 214
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-4.47664064e-07, -6.79364192e-08, -2.76864421e-06, ...,
        -2.22185612e-08, -2.29140042e-07, -3.76006824e-07],
       [ 1.46871230e-06, -1.16599395e-07, -6.41232191e-06, ...,
        -2.89472325e-07, -8.50607307e-07, -4.11914016e-07],
       [ 3.74957239e-08, -3.42635531e-07, -1.33526271e-07, ...,
         3.31317665e-07, -4.34709506e-07, -6.83452299e-07]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([-1.41928676e-05,  3.47124751e-06, -2.71401277e-05,  6.

...

#bot_tweets: 94 #gen_tweets: 94
Input count: 188, 188
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.4578218e-05, -4.5628181e-05, -1.9334882e-05, ...,
         0.0000000e+00,  8.7091521e-06,  9.4937459e-06],
       ...,
       [-7.4033011e-07,  1.7748025e-06,  4.1135124e-07, ...,
         8.3953211e-07,  6.7108533e-07,  1.2533544e-06],
       [-1.1844367e-05,  3.9171522e-05,  8.2824754e-06, ...,
         1.0066068e-06, -6.7958858e-06, -1.0285000e-06],
       [-7.9681431e-06,  3.4279285e-05,  1.0595219e-05, ...,
         2.0546105e-07, -8.3489876e-06, -1.7218058e-06]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([ 1.37041599e-04, -6.02939748e-04, -6.34094831e-05,  1.76630259e-04,
        0.00000000e+00, 

#bot_tweets: 103 #gen_tweets: 103
Input count: 206, 206
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 9.6159408e-07, -2.4731917e-06, -3.8851699e-06, ...,
        -3.8466510e-07,  0.0000000e+00,  1.5714663e-06],
       [ 1.2191217e-06,  2.7630773e-07,  1.7520865e-06, ...,
        -1.2115586e-06,  0.0000000e+00, -2.5284996e-06],
       [ 1.2166101e-06, -7.2403287e-07, -1.5267509e-06, ...,
        -3.5501296e-07,  0.0000000e+00, -4.4085675e-07]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([-9.88390530e-05,  7.01524623e-05,  3.70691232e-05,  5.79642256e-06,
       -5.82622379e-05

#bot_tweets: 105 #gen_tweets: 105
Input count: 210, 210
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  2.0255886e-06, ...,
        -4.9332971e-06,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.2155635e-06, -1.1348059e-05, -1.3604651e-05, ...,
         1.7770464e-05, -3.3891424e-06, -1.3358696e-05],
       [-7.9621202e-07,  3.4659224e-06,  2.4433484e-05, ...,
        -3.0579693e-05, -1.7653596e-07,  2.7443699e-05],
       [ 1.9906027e-07, -1.9298577e-06,  2.5127479e-06, ...,
        -3.9283636e-06, -7.8364701e-07,  3.3587485e-06]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([-8.25380903e-06,  7.56599911e-05,  1.04479033e-04,  6.53601955e-06,
        2.26690099e-06

#bot_tweets: 110 #gen_tweets: 110
Input count: 220, 220
Curr gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00,  2.7968510e-05,  0.0000000e+00, ...,
        -5.0910448e-06,  0.0000000e+00,  1.3976763e-05],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.7024446e-05,  6.4806613e-06, -1.2216311e-06, ...,
         4.2174224e-06, -8.2018851e-06,  3.9272982e-06],
       [-8.7212748e-06, -3.1038057e-06, -3.8186215e-07, ...,
        -1.6371484e-06,  1.8136983e-06,  2.1879396e-06],
       [ 1.6447515e-05,  1.1176351e-05,  1.3551119e-06, ...,
        -1.3212489e-06, -4.5045745e-06,  6.7818205e-06]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([ 3.76830576e-04,  2.57511769e-04,  4.36604139e-04, -1.13600116e-04,
       -8.10311292e-04

Sum of gd: [<tf.Tensor: shape=(38, 500), dtype=float32, numpy=
array([[ 0.0000000e+00, -1.1687484e-06,  0.0000000e+00, ...,
        -3.6305224e-07,  0.0000000e+00,  1.6548127e-07],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -1.1311359e-06,  0.0000000e+00, ...,
        -9.6262352e-07,  8.0354221e-07,  0.0000000e+00],
       ...,
       [-1.9126048e-07,  4.3795867e-06, -7.4880290e-08, ...,
         1.0497497e-06, -5.8225637e-06, -1.1717495e-06],
       [-1.9072915e-07,  5.9109561e-07, -4.3454683e-08, ...,
         2.2985587e-06, -3.3951014e-06,  1.0577437e-06],
       [-2.2686422e-09, -2.6719797e-06, -4.4141423e-08, ...,
         5.9597397e-07, -1.6506146e-07, -1.8497883e-07]], dtype=float32)>, <tf.Tensor: shape=(500,), dtype=float32, numpy=
array([ 3.10155974e-06, -1.16721128e-06,  2.59644025e-07,  1.94834479e-06,
        1.24073762e-04, -1.67848426e-04, -8.97357168e-05, -1.03051434e-05,
 

In [21]:
spark.stop()